### Selecionando os dados de cartas válidas no commander

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# Função para buscar dados da API do Scryfall
def fetch_all_data(query):
    url = f"https://api.scryfall.com/cards/search?{query}"
    response = requests.get(url)
    if response.status_code != 200:
        return []

    data = response.json()
    cards = data['data']

    while data.get('has_more'):
        next_page_url = data['next_page']
        response = requests.get(next_page_url)
        if response.status_code != 200:
            break
        data = response.json()
        cards.extend(data['data'])

    return cards

# Buscar todas as cartas válidas no formato commander
query = "q=legal:commander"

cards = fetch_all_data(query)

# Convertendo para DataFrame
df = pd.DataFrame(cards)

In [2]:
df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id',
       'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri',
       'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris',
       'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors',
       'color_identity', 'keywords', 'legalities', 'games', 'reserved', 'foil',
       'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation',
       'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri',
       'scryfall_set_uri', 'rulings_uri', 'prints_search_uri',
       'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id',
       'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame',
       'full_art', 'textless', 'booster', 'story_spotlight', 'edhrec_rank',
       'preview', 'prices', 'related_uris', 'purchase_uris', 'power',
       'toughness', 'all_parts', 'frame_effects', 'security_stamp',
       'promo_types', 'mtgo_foil_id'

In [3]:
len(df)

27279

In [4]:
fhdh

NameError: name 'fhdh' is not defined

In [ ]:
# Gráfico de Barras para Tipos de Criatura
def extract_main_type(type_line):
    parts = type_line.split('—')
    if len(parts) > 1:
        return parts[1].strip().split()[0]
    return None

df['main_type'] = df['type_line'].apply(extract_main_type)

# Remover duplicatas e manter apenas uma palavra por tipo
unique_main_types = df['main_type'].drop_duplicates()

# Nuvem de Palavras para texto carta
all_text = ' '.join(unique_main_types.dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Nuvem de Palavras para texto carta
all_text = ' '.join(df['oracle_text'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
df['keywords'].head(20)

In [ ]:
def separate_keywords(keywords):
    if not keywords:
        return []
    return [kw.strip() for keyword in keywords for kw in keyword.split(',')]

df['keywords'] = df['keywords'].apply(separate_keywords)
df = df.explode('keywords')

# Remover duplicatas nas palavras-chave
keywords_set = set(df['keywords'].dropna())
individual_keywords = ' '.join(keywords_set)

# Gerar a nuvem de palavras individual
wordcloud_individual = WordCloud(width=800, height=400, background_color='white').generate(individual_keywords)

# Plotar a nuvem de palavras individual
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_individual, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras Individual')
plt.show()


In [ ]:
# Filtrar e ordenar pelo rank do EDHREC
df = df[df['edhrec_rank'].notnull()]  # Filtrar cartas sem rank EDHREC
df = df.sort_values('edhrec_rank').head(10)  # Top 10 cartas

# Mostrar imagens das cartas
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for i, (index, row) in enumerate(df.iterrows()):
    image_uris = row['image_uris']
    if isinstance(image_uris, dict):
        img_url = image_uris.get('normal')
        if img_url:
            response = requests.get(img_url)
            img = Image.open(BytesIO(response.content))
            axes[i].imshow(img)
            axes[i].axis('off')
            axes[i].set_title(f"{row['name']} (Rank: {row['edhrec_rank']})", fontsize=12)
        else:
            axes[i].axis('off')
            axes[i].set_title(f"{row['name']} (Rank: {row['edhrec_rank']})\nImagem não disponível", fontsize=12)
    else:
        axes[i].axis('off')
        axes[i].set_title(f"{row['name']} (Rank: {row['edhrec_rank']})\nImagem não disponível", fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
edhrec_data

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from io import BytesIO

# Função para buscar dados da API do Scryfall
def fetch_all_commanders():
    url = "https://api.scryfall.com/cards/search"
    query = "q=is:commander"
    response = requests.get(f"{url}?{query}")
    if response.status_code != 200:
        return []

    data = response.json()
    cards = data['data']

    while data.get('has_more'):
        next_page_url = data['next_page']
        response = requests.get(next_page_url)
        if response.status_code != 200:
            break
        data = response.json()
        cards.extend(data['data'])

    return cards

# Buscar todas as cartas de comandante
commanders = fetch_all_commanders()

# Converter os dados para DataFrame
df_commanders = pd.DataFrame(commanders)

# Explodir colunas de listas para facilitar a análise
df_commanders = df_commanders.explode('colors')
df_commanders = df_commanders.explode('keywords')

# Analisar a distribuição das cores dos comandantes
color_distribution = df_commanders['colors'].value_counts()

# Analisar a distribuição das palavras-chave dos comandantes
keyword_distribution = df_commanders['keywords'].value_counts()

# Configurações de estilo
sns.set(style="whitegrid")

# Gráfico de barras da distribuição das cores dos comandantes
plt.figure(figsize=(10, 6))
sns.barplot(x=color_distribution.index, y=color_distribution.values)
plt.title('Distribuição das Cores dos Comandantes')
plt.xlabel('Cor')
plt.ylabel('Quantidade')
plt.show()

# Gráfico de barras da distribuição das palavras-chave dos comandantes
plt.figure(figsize=(10, 6))
sns.barplot(x=keyword_distribution.index, y=keyword_distribution.values)
plt.title('Distribuição das Palavras-chave dos Comandantes')
plt.xlabel('Palavras-chave')
plt.ylabel('Quantidade')
plt.show()

# Gerar a nuvem de palavras combinada
all_keywords_combined = ' '.join(df_commanders['keywords'].dropna())
wordcloud_combined = WordCloud(width=800, height=400, background_color='white').generate(all_keywords_combined)

# Gerar a nuvem de palavras individual
individual_keywords = ' '.join(df_commanders['keywords'].dropna().explode())
wordcloud_individual = WordCloud(width=800, height=400, background_color='white').generate(individual_keywords)

# Plotar a nuvem de palavras combinada
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_combined, interpolation='bilinear')


# Plotar a nuvem de palavras individual
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_individual, interpolation='bilinear')



In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from io import BytesIO

# Função para buscar dados da API do Scryfall
def fetch_all_commander_legal_cards():
    url = "https://api.scryfall.com/cards/search"
    query = "q=legal:commander"
    response = requests.get(f"{url}?{query}")
    if response.status_code != 200:
        return []

    data = response.json()
    cards = data['data']

    while data.get('has_more'):
        next_page_url = data['next_page']
        response = requests.get(next_page_url)
        if response.status_code != 200:
            break
        data = response.json()
        cards.extend(data['data'])

    return cards

# Buscar todas as cartas legais no formato Commander
cards = fetch_all_commander_legal_cards()

# Converter os dados para DataFrame
df_cards = pd.DataFrame(cards)

# Filtrar apenas comandantes
df_commanders = df_cards[df_cards['type_line'].str.contains('Legendary')]

# Explodir colunas de listas para facilitar a análise
df_commanders = df_commanders.explode('colors')
df_commanders = df_commanders.explode('keywords')

# Analisar a distribuição das cores dos comandantes
color_distribution = df_commanders['colors'].value_counts()

# Analisar a distribuição das palavras-chave dos comandantes
keyword_distribution = df_commanders['keywords'].value_counts()

# Configurações de estilo
sns.set(style="whitegrid")

# Gráfico de barras da distribuição das cores dos comandantes
plt.figure(figsize=(10, 6))
sns.barplot(x=color_distribution.index, y=color_distribution.values)
plt.title('Distribuição das Cores dos Comandantes')
plt.xlabel('Cor')
plt.ylabel('Quantidade')
plt.show()

# Gráfico de barras da distribuição das palavras-chave dos comandantes
plt.figure(figsize=(10, 6))
sns.barplot(x=keyword_distribution.index, y=keyword_distribution.values)
plt.title('Distribuição das Palavras-chave dos Comandantes')
plt.xlabel('Palavras-chave')
plt.ylabel('Quantidade')
plt.show()

# Gerar a nuvem de palavras combinada
all_keywords_combined = ' '.join(df_commanders['keywords'].dropna())
wordcloud_combined = WordCloud(width=800, height=400, background_color='white').generate(all_keywords_combined)

# Gerar a nuvem de palavras individual
individual_keywords = ' '.join(df_commanders['keywords'].dropna().explode())
wordcloud_individual = WordCloud(width=800, height=400, background_color='white').generate(individual_keywords)

# Plotar a nuvem de palavras combinada
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_combined, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras Combinada')
plt.show()

# Plotar a nuvem de palavras individual
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_individual, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras Individual')
plt.show()
